# Example of how to derive conductances from SSUSI and use that in Lompe

First import packages. I got dataloader function to work with the Iridium dataset with the following combination of packages and python version:  
python==3.7  
shapely.__version__==1.7.1  
astropy.__version__==4.2.1  


In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from secsy import cubedsphere
from lompe import conductance
import apexpy
import lompe
import lompe.data_tools.dataloader as dataloader
import xarray as xr
from lompe.utils import sunlight
import matplotlib.pyplot as plt

Select a date to work with. This script will assume that you are logged on the the BCSS dropbox account, where it will fetch the data. One excemption is the SuperMAG data. The new python-API should be implemented to download the data directly from the supermag server.


In [2]:
event = '2013-06-01'

Specify Dropbox path and path to where temp dataset files are stored. 

In [3]:
basepath = '/Users/jone/BCSS-DAG Dropbox/'
tempfile_path='./data/'

Load the data and make the temp files

In [ ]:
ampere    = pd.read_pickle(dataloader.read_iridium(event, basepath=basepath,tempfile_path=tempfile_path))
superdarn  = pd.read_pickle(dataloader.read_sdarn(event, basepath=basepath,tempfile_path=tempfile_path))
supermag = pd.read_pickle(dataloader.read_smag(event, basepath=basepath,tempfile_path=tempfile_path))
ssusi = xr.load_dataset(dataloader.read_ssusi(event, basepath=basepath,tempfile_path=tempfile_path))


Select time to work with


In [ ]:
ii = 29 # number of dmsp pass to select for condutance models
time = pd.to_datetime(ssusi['date'].values[ii])
#time = dt.datetime(int(event[0:4]),int(event[5:7]),int(event[8:10]),22,48)
DT = dt.timedelta(seconds = 5 * 60) # will select data from time +- DT


Set up grid

In [ ]:
position = (-80, 82) # lon, lat
orientation = (-0.1, 1) # east, north
L, W, Lres, Wres = 10000e3, 6000e3, 100.e3, 100.e3 # dimensions and resolution of grid (L, Lres are along orientation vector)
grid = cubedsphere.CSgrid(cubedsphere.CSprojection(position, orientation), L, W, Lres, Wres, R = 6481.2e3)

## Make the conductance model

1) First estimate condutance from EUV radiation. Construct with smaller weight to reduce impact where we have SSUSI coverage

In [ ]:
sza = sunlight.sza(grid.lat, grid.lon, time)
EUV_hall, EUV_pedersen = conductance.EUV_conductance(sza, 100, 'hp', calibration = 'MoenBrekke1993')
sh_euv = lompe.Data(EUV_hall, # data values
                     np.vstack((np.ravel(grid.lon), np.ravel(grid.lat))), # coordinates
                     datatype = 'Hall', weights = np.ones(grid.lon.flatten().shape[0])*0.5)
sp_euv = lompe.Data(EUV_pedersen, # data values
                     np.vstack((np.ravel(grid.lon), np.ravel(grid.lat))), # coordinates
                     datatype = 'Pedersen', weights = np.ones(grid.lon.flatten().shape[0])*0.5)

2) Use SSUSI to estimate conductance from precipitation:

In [ ]:
a = apexpy.Apex(time.year)
glat, glon = a.convert(ssusi.isel(date=ii).mlat, ssusi.isel(date=ii).mlt,'mlt','geo',height=110, datetime=time)
inside_px = grid.ingrid(glon, glat, ext_factor=1)
sh = np.array(ssusi.isel(date=ii).SH).flatten()
sp = np.array(ssusi.isel(date=ii).SP).flatten()
use_sh = np.isfinite(sh) & inside_px.flatten()
use_sp = np.isfinite(sp) & inside_px.flatten()
sza_ssusi = sunlight.sza(glat, glon, pd.to_datetime(ssusi['date'].values[ii]))
EUV_hall_ssusi, EUV_pedersen_ssusi = conductance.EUV_conductance(sza_ssusi, 100, 'hp', calibration = 'MoenBrekke1993')
sh_ssusi = lompe.Data(sh[use_sh] + EUV_hall_ssusi[use_sh], # data values
                     np.vstack((glon.flatten()[use_sh], glat.flatten()[use_sh])), # coordinates
                     datatype = 'Hall')
sp_ssusi = lompe.Data(sp[use_sp] + EUV_pedersen_ssusi[use_sp], # data values
                     np.vstack((glon.flatten()[use_sp], glat.flatten()[use_sp])), # coordinates
                     datatype = 'Pedersen')

3) Initialize the model, add data, and run inversion

In [ ]:
cmod = lompe.Cmodel(grid,spline_smoothing = 0)
cmod.add_data(sh_ssusi, sp_ssusi, sh_euv, sp_euv)  # add datasets
cmod.run_inversion()

4) Explore the conductance model result and compare to data

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize = (14, 10))
# Plot grid and coastlines:
for ax in axs:
    ax.set_axis_off()
    for lon, lat in grid.get_grid_boundaries():
        xi, eta = grid.projection.geo2cube(lon, lat)
        ax.plot(xi, eta, color = 'grey', linewidth = .4)
    xlim, ylim = ax.get_xlim(), ax.get_ylim()
    for cl in grid.projection.get_projected_coastlines():
        ax.plot(cl[0], cl[1], color = 'C0')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
ax = axs[0]
#Plot ssusi on the projected map
#Reduce size of ssusi image to at least not exceed the CS grid face
__x = 30 #30
__y = 290 #300
ax.pcolormesh(grid.xi_mesh, grid.eta_mesh,sh_euv.values.reshape(grid.lon.shape), vmax=40, vmin=0)
xi, eta = grid.projection.geo2cube(glon, glat)
ax.pcolormesh(xi[__x:__y,__x:__y],eta[__x:__y,__x:__y],(sh+EUV_hall_ssusi).reshape(xi.shape)[__x:__y,__x:__y], vmax=40, vmin=0)
ax.text(0.3,0.99,'Input data + EUV', transform=ax.transAxes)
#Plot the result of inversion
ax = axs[1]
Gc = cmod.conductance_matrix(lon = grid.lon_mesh, lat = grid.lat_mesh)
hall_c = Gc.dot(cmod.m['hall'])
ax.pcolormesh(grid.xi_mesh, grid.eta_mesh, np.exp(hall_c.reshape(grid.lon_mesh.shape)), vmax=40, vmin=0)
ax.text(0.3,0.99,'Conductance inversion', transform=ax.transAxes)

## Electric field model (Lompe)

1) Prepare datasets

In [ ]:
# prepare ampere
amp = ampere[(ampere.time >= time - DT) & (ampere.time <= time + DT)].dropna()
B = np.vstack((amp.B_e.values, amp.B_n.values, amp.B_r.values))
coords = np.vstack((amp.lon.values, amp.lat.values, amp.r.values))
amp_data = lompe.Data(B * 1e-9, coords, datatype = 'space_mag_full', scale = 200 * B.size * 1e-9)

# prepare supermag
sm = supermag[time-DT:time+DT].dropna()
B = np.vstack((sm.Be.values, sm.Bn.values, sm.Bu.values))
coords = np.vstack((sm.lon.values, sm.lat.values))
sm_data = lompe.Data(B * 1e-9, coords, datatype = 'ground_mag', scale = 100 * B.size * 1e-9)

# prepare superdarn
sd = superdarn.loc[(superdarn.index >= time - DT) & (superdarn.index <= time+ DT)].dropna()
vlos = sd['vlos'].values
coords = np.vstack((sd['glon'].values, sd['glat'].values))
los  = np.vstack((sd['le'].values, sd['ln'].values))
sd_data = lompe.Data(vlos, coordinates = coords, LOS = los, datatype = 'convection', scale = (100000 * len(vlos))**2 )


2) Initialize model and add datasets, and run inversion

In [ ]:
emod = lompe.Emodel(grid, (cmod.hall, cmod.pedersen))
emod.add_data(amp_data, sm_data, sd_data)
emod.run_inversion(l1 = 2, l2 = 0)

Lompeplot

In [ ]:
lompe.lompeplot(emod, include_data = True, time = time, apex = a)